In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install ohmeow-blurr -q
!pip install datasets -q
!pip install bert-score -q

In [2]:
cd /content/drive/Shareddrives/KLUE Summarization/

/content/drive/Shareddrives/KLUE Summarization


In [7]:
import json
import pandas as pd


def read_jsonl(path):
  DATA_DIR = "문서요약 텍스트_unzip"
  with open(DATA_DIR + path , 'r') as json_file:
      json_list = list(json_file)

  trains = []
  for json_str in json_list:
      line = json.loads(json_str)
      trains.append(line)
  df = pd.DataFrame(trains)
  return df

train_df = read_jsonl('/1.Training/신문기사_1.train.jsonl/train.jsonl')
val_df = read_jsonl('/2.Validation/신문기사_1.vaild.jsonl/vaild.jsonl')


def clean(df):
    article = []

    for i in range(df.shape[0]):
        text = ''
        for j in range(len(df['article_original'][i])):
            text += " " + df['article_original'][i][j]
        article.append(text)

    df['article'] = article

    return df
clean(train_df)
df = train_df

In [8]:
import datasets
import pandas as pd
from fastai.text.all import *
from transformers import *

from blurr.data.all import *
from blurr.modeling.all import *

pretrained_model_name = "hyunwoongko/kobart"
hf_arch, hf_config, hf_tokenizer, hf_model = BLURR.get_hf_objects(pretrained_model_name, 
                                                                  model_cls=BartForConditionalGeneration)

hf_arch, type(hf_config), type(hf_tokenizer), type(hf_model)


text_gen_kwargs = default_text_gen_kwargs(hf_config, hf_model, task='summarization')
text_gen_kwargs['max_length'] = 80
text_gen_kwargs['min_length'] = 20

text_gen_kwargs


hf_batch_tfm = HF_Seq2SeqBeforeBatchTransform(hf_arch, hf_config, hf_tokenizer, hf_model, 
                                              max_length=650, max_tgt_length=130, text_gen_kwargs=text_gen_kwargs)

blocks = (HF_Seq2SeqBlock(before_batch_tfm=hf_batch_tfm), noop)

dblock = DataBlock(blocks=blocks, get_x=ColReader('article'), get_y=ColReader('abstractive'), splitter=RandomSplitter())


dls = dblock.dataloaders(df, bs=2)
b = dls.one_batch()

seq2seq_metrics = {
        'rouge': {
            'compute_kwargs': { 'rouge_types': ["rouge1", "rouge2", "rougeL"], 'use_stemmer': True },
            'returns': ["rouge1", "rouge2", "rougeL"]
        },
        'bertscore': {
            'compute_kwargs': { 'lang': 'en' },
            'returns': ["precision", "recall", "f1"]
        }
    }

model = HF_BaseModelWrapper(hf_model)
learn_cbs = [HF_BaseModelCallback]
fit_cbs = [HF_Seq2SeqMetricsCallback(custom_metrics=seq2seq_metrics)]

learn = Learner(dls, 
                model,
                opt_func=ranger,
                loss_func=CrossEntropyLossFlat(),
                cbs=learn_cbs,
                splitter=partial(seq2seq_splitter, arch=hf_arch)).to_fp16()

learn.create_opt() 
learn.freeze()

learn.lr_find()

b = dls.one_batch()
preds = learn.model(b[0])

Downloading:   0%|          | 0.00/2.17k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.93k [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/torch/cuda/amp/autocast_mode.py:120: UserWarning: torch.cuda.amp.autocast only affects CUDA ops, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.autocast only affects CUDA ops, but CUDA is not available.  Disabling.")
/usr/local/lib/python3.7/dist-packages/torch/cuda/amp/grad_scaler.py:115: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")


KeyboardInterrupt: ignored

In [14]:
b = dls.one_batch()
preds = learn.model(b[0])

In [ ]:
import datasets
import pandas as pd
from fastai.text.all import *
from transformers import *

from blurr.data.all import *
from blurr.modeling.all import *

In [4]:
org_input = """
크게 막 배우는 건 없습니다. 행렬 말고는 대부분 고등학교 때 배웠던 부분입니다...
그런데 중간중간 나오는 숙제가 있는데 쉽다고 대충 푸시면 생각보다 많이 까입니다...
그리고 칼절평입니당....이남경 교수님 경영수학은 다른 경영수학 수업에 비해서 깊은 수학을 다루진 않는다고 들었고, 실제로도 그랬던 거 같습니다. 고등학교 수학으로 커버 되는 부분이 꽤 많습니다. 그래도 기말 부분에는 새로운 내용들이 많이 추가되니까 너무 날로 먹을 생각만 하면 안 될 거 같습니다. 강의력은 그냥 보통 정도 됩니다. 그렇게 나쁘지도 않고 그렇게 좋지도 않습니다. 과제는 총 5번 나왔고, 제출 한다고 만점이 아니라 답까지 맞아야 합니다. 점수를 그렇게 많이 까진 않지만 칼절평이라서 조금의 감점이 나중에 꽤 크게 작용합니다. 중간고사는 쉬웠고 기말고사는 어려웠는데 채점을 되게 후하게 해주십니다. 아는 부분이라도 다 적으면 꽤 많은 점수를 얻을 수 있습니다. 앞서 말했듯 칼절평입니다. 0.5점 차이도 안 봐줍니다.거의 고등수학(문과 기준) 내용이라 보시면 돼요. 아 요즘은 개정됐죠? 지금 기준으로는 잘 모르겠네요.
과제 채점을 되게 빡세게 하세요. 칼절평인 만큼 정말 과제, 시험 모든 부분에서 방심하시면 안 돼요...
콜라보레이트 실시간이지만 녹화도 해주셔서 잘 참고했어요. 아 그런데 중간중간 출석 부르시기도 합니다.



그리고 개인적으로는,, 교수님께서 설명을 너무 느리게 하셔서,, 움,, 조금 답답했습니다 ㅜㅜ
그렇게 어려운 내용도 아니고 고딩 때 거의 다 배운 내용인데 ㅜㅜ정말 칼같은 절평이었습니다. 내용 자체는 크게 어려운 부분은 별로 없어요. 과제가 좀 있긴한데 그냥 문제푸는겁니다. 근데 절평이 너무 칼같아서 꽤 공들여서 해야합니다. 저를 포함해서 이번에 썰리신분 좀 많은것같아요뭔가 고등학교때 수학 실력과는 상관 없는 강의입니다. Linear programming을 정복하는 사람은 좋은 학점을 받을 수 있을 것입니다. 성적은 칼채점 하십니다.수업 좋았습니다. 95 칼절평이라고 해도 채점을 ㄹㅇ 후하게 하십니다. 그래서 94.5로 에이쁠 못받았다? 그건 다른 수업으로 치면 그냥 89점 받은거에요. 절평 너무 걱정 안하시고 들으면 될 것 같습니다. 난이도도 매우 쉽습니다.꿀강이라고 유명"했던" 강의였지만, 이번 학기에 흑화하셨습니다.
지난 학기에는 과제도 없었다고 했는데 과제도 5번 꼬박꼬박 내시고 난이도도 중간 이전까지는 괜찮나 중간 이후부터는 쉽지 않습니다. 칼절평이기 때문에 과제에서 점수가 깎이면 정말 피눈물 납니다.

지난 학기에 학생들이 온라인 시험이라 프로그램을 돌려서 시험을 봤다는데, 이번 학기부터는 락다운 브라우저 때문에 그런 꼼수는 통하지 않습니다. 시험 문제들이 어렵지는 않으나, simplex table을 직접 그려서 계산하는 게 상당히 시간을 많이 잡아 먹으니 다른 문제들에서 시간을 아끼셔야 할 겁니다. 이름은 simplex지만 전혀 simple하지는 않고 심지어 설명도 제대로 해 주시지 않습니다. 독학을 해서라도 완전히 푸는 방법을 익히시고 가세요.

참고로 simplex table 문제는 중간 기말에 모두 마지막 문제로 출제됐습니다. initial table까지만 그려도 부분 점수는 주시니 정 어려우면 initial table그리는 법까지만 외우세요.
기말고사에는 뜬금없이 T/F 및 증명 문제가 나왔으니, 이 부분도 염두에 두고 수강 신청해 주세요.

학점은 정말 베일 듯한 칼절평입니다. 0.25점 차이로 A가 나와서 입맛이 쓰네요.

수업 난이도가 어렵지는 않습니다.칼 절평.
출석 랜덤하게 세 번?정도 부른듯
중간고사 전 범위는 엄청 쉽고 시험도 쉬웠음. 기말고사가 어려웠음. 중간고사에서 1점 깎여서 기말도 잘 보면 에이쁠 받을 수 있을 줄 알았는데 기말 시험이 어렵게 느껴져서 좀 못 보고 에이 나옴.학점 절평이고 중학교 수학과목 느낌입니다. 그리고 과제를 잘 하셔야해요. 절평이라서 과제에서 깍이면 잘 커버안됩니다. 출첵 잘하시니까 지각 하시면 안됩니다. 교수님은 상당히 친절하세요정말 칼 같이 절평 하십니다

미리 밝히지만 저는 총점 89.9로 비쁠 받았습니다(?)

출석 10 중간30 기말 30 과제30(총6번) 이었습니다

중간이랑 기말은 모두 평균이상으로 잘 봤는데
과제에서 짜잘짜잘하게 점수가 많이 까이더니 총점이 89.9가 나왔습니다

근데 칼 같이 비쁠 주시네요 ㅎㅎㅎㅎㅎㅎㅎ
이번 학기에서 가장 슬픈 강의였습니다
"""

In [5]:
model = "news_summary_200000.pkl"

In [6]:
cd /content/drive/MyDrive/KLUE

/content/drive/MyDrive/KLUE


In [7]:
inf_learn = load_learner(fname=model)

NameError: ignored

In [8]:
inf_learn.blurr_generate(org_input)

NameError: ignored